Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy.random
import random
from random import randint
import math
import copy

Разные функции для использования в Node

In [ ]:
identity = lambda x: x
isrlu = lambda x: (x / (math.sqrt(1 + x**2)) if (x < 0) else x)
bent_identity = lambda x: ((math.sqrt(x**2 + 1) - 1) / 2) + x
th = lambda x: math.tanh(x)
arctg = lambda x: math.atan(x)
softsign = lambda x: x / (1 + math.fabs(x))
isru = lambda x: x / math.sqrt(1 + x**2)
relu = lambda x: 0 if (x < 0) else x
leaky_relu = lambda x: (x * 0.01) if (x < 0) else x
softplus = lambda x: np.log(1 + np.exp(x))
step = lambda x: 0 if (x < 0) else 1
logistic = lambda x: 1 / (1 + np.exp(-x))
sinus = lambda x: math.sin(x)
cosinus = lambda x: math.cos(x)
sigmoid = lambda x: (1 / (1 + np.exp(-x))) * x
gauss = lambda x: np.exp(-(x**2))
sinc = lambda x: 1 if (x == 0) else math.sin(x) / x
inverse = lambda x: -x
absolute = lambda x: math.fabs(x)

Массив функций

In [ ]:
# functions = [identity, isrlu, bent_identity, th, arctg, softsign, isru, relu, leaky_relu,
#              softplus, step, logistic, sinus, cosinus, sigmoid, gauss, sinc, inverse, absolute]
colors = ['black', 'deeppink', 'orange', 'green', 'blue', 'yellow', 'violet', 'salmon', 'pink', 'orangered', 'lime', 'olive', 'teal', 'lightcyan']
functions = [identity, step, sinus, cosinus, gauss, th, sigmoid, inverse, absolute, relu]
print(len(functions))

10


Класс Node.
 
Параметры: 
layer - слой, на который будет помещен данный экземпляр
foo - функция активации

Методы:
run - добавляет к каждому ноду из next собственную сумму, пропущенную через функцию активации и умноженную на weight


In [ ]:
class Node:
    """
    Класс Node.
    Параметры: 
    layer - слой, на который будет помещен данный экземпляр
    foo - функция активации
    """
    def __init__(self, layer=0, foo=identity):
        self.layer = layer
        self.sum = 0
        self.foo = foo
        self.next = []
        self.pos = 0
    
    def run(self, weight):
        """
        Параметры:
        weight - вес
        Добавляет к каждому ноду из next собственную сумму, пропущенную через функцию активации и умноженную на weight
        """
        try:
            value = self.foo(self.sum) * weight
        except OverflowError:
            value = int(0)
        for one in self.next:
            one.sum += value

In [ ]:
a = 100
b = a
b += 1
print(b, a)

101 100


In [ ]:
class Neyro:
    def __init__(self):
        self.connections = 64
        self.nodes = []
        self.max_layer = 1
        self.score = 64
        self.games_played = 0

        # filling an array of nodes
        for layer_ in range(2):
            for shhh in range(64):
                self.nodes.append(Node(layer_))

        # connect each output node with random amount of input nodes
        for i in range(64):
            temp = randint(0, 63)
            if self.nodes[64 + i] not in self.nodes[temp].next:
                self.nodes[temp].next.append(self.nodes[64 + i])
            
    def add_node(self):
        self.connections += 1
        self.score += 1

        # chose first node
        first = self.nodes[randint(0, len(self.nodes) - 1)]
        while first.layer == self.max_layer or len(first.next) == 0:
            first = self.nodes[randint(0, len(self.nodes) - 1)]

        # chose second node
        second = first.next[randint(0, len(first.next) - 1)]
        # assert second in self.nodes

        third = Node(foo=random.choice(functions))
        if second.layer - first.layer == 1:
            c = second.layer
            for node in self.nodes:
                if node.layer >= c:
                    node.layer += 1
            third.layer = c
            self.max_layer += 1

        else:
            third.layer = first.layer + 1

        third.next.append(second)
        first.next.remove(second)
        first.next.append(third)
        self.nodes.append(third)

    def add_connection(self):
        self.connections += 1
        self.score += 1

        first = self.nodes[randint(0, len(self.nodes) - 1)]

        second = self.nodes[randint(0, len(self.nodes) - 1)]
        while second.layer == first.layer or (second in first.next) or (first in second.next):
            second = self.nodes[randint(0, len(self.nodes) - 1)]

        if second.layer > first.layer:
            first.next.append(second)
        else:
            second.next.append(first)


    def change_activation(self):
        if self.max_layer == 1:
            pass
        else:
            temp = self.nodes[randint(0, len(self.nodes) - 1)]
            while temp.layer == 0 or temp.layer == self.max_layer:
                temp = self.nodes[randint(0, len(self.nodes) - 1)]
            temp.foo = random.choice(functions)

    def predict(self, position, weight):
        for star in range(64):
            # print(len(self.nodes), len(position), star)
            self.nodes[star].sum = position[star]

        for layer in range(self.max_layer):
            for node in self.nodes:
                if node.layer == layer:
                    node.run(weight)

        result = []
        counter = 0
        for node in self.nodes:
            if node.layer == self.max_layer:
                result.append(node.sum)
                counter += 1
        return result

In [ ]:
# pos = [1] * 64
# b = Neyro()
# for t in range(100):
#     r = randint(1, 3)
#     if r == 1:
#         b.add_node()
#     if r == 2:
#         b.add_connection()
#     if r == 3:
#         b.change_activation()

# print(b.predict(pos, 2))
# print(len(b.nodes) - 128)
# print(b.connections)

# fig, ax = plt.subplots(1)
# pos = 0

# for layer in range(b.max_layer + 1):
#     for node in b.nodes:
#         if node.layer == layer:
#             ax.scatter(layer, pos, c=colors[functions.index(node.foo)])
#             node.pos = pos
#             pos += 1
#     pos = 0

# for node in b.nodes:
#     for one in node.next:
#         ax.plot((node.layer, one.layer), (node.pos, one.pos), linewidth=0.2, color='blue')
# # plt.figure(figsize=(32, 18))
# plt.show()

In [ ]:
def line_to_array(pos):
    position = (([0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]),
                ([0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]),
                ([0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]),
                ([0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]))
    
    counter = 0

    for k in range(4):
        for i in range(4):
            for j in range(4):
                position[k][i][j] = pos[counter]
                counter += 1
                
    return position

In [ ]:
def check_win(pos):
    if 0 not in pos:
        return True

    position = line_to_array(pos)
    if ((position[0][0][0] == position[1][1][1] == position[2][2][2] == position[3][3][3] != 0)
        | (position[3][0][0] == position[2][1][1] == position[1][2][2] == position[0][3][3] != 0)
        | (position[0][0][3] == position[1][1][2] == position[2][2][1] == position[3][3][0] != 0)
        | (position[0][3][0] == position[1][2][1] == position[2][1][2] == position[3][0][3] != 0)):
        return True

    for k in range(4):
        for i in range(4):
            for j in range(4):
                if ((position[k][0][0] == position[k][1][1] == position[k][2][2] == position[k][3][3] != 0)
                    | (position[k][3][0] == position[k][2][1] == position[k][1][2] == position[k][0][3] != 0)
                    | (position[k][0][j] == position[k][1][j] == position[k][2][j] == position[k][3][j] != 0)
                    | (position[k][i] == [1, 1, 1, 1]) | (position[k][i] == [-1, -1, -1, -1])
                    | (position[0][i][j] == position[1][i][j] == position[2][i][j] == position[3][i][j] != 0)
                    | (position[0][i][0] == position[1][i][1] == position[2][i][2] == position[3][i][3] != 0)
                    | (position[3][i][0] == position[2][i][1] == position[1][i][2] == position[0][i][3] != 0)
                    | (position[0][0][j] == position[1][1][j] == position[2][2][j] == position[3][3][j] != 0)
                    | (position[3][0][j] == position[2][1][j] == position[1][2][j] == position[0][3][j] != 0)):
                    return True

    return False

In [ ]:
pos = [0] * 64

In [ ]:
%%timeit
check_win(pos)

1000 loops, best of 5: 246 µs per loop


In [ ]:
def game(player1, player2, weight):
    position = np.zeros(64, dtype=int)

    
    while not check_win(position):

        # X - move
        res1 = player1.predict(position, weight)
        while position[np.argmax(res1)] != 0:
            res1[np.argmax(res1)] = float('-inf')
        position[np.argmax(res1)] = 1
        if check_win(position): # player1 won
            player1.score += 1000
            player2.score -= 1000
            return

        # O - move
        res2 = player2.predict((-1) * position, weight)
        while position[np.argmax(res2)] != 0:
            res2[np.argmax(res2)] = float('-inf')
        position[np.argmax(res2)] = -1
    
    player2.score += 1000
    player1.score -= 1000
    return

In [ ]:
a = [float('-inf'), 1]
b = absolute(a[0])
np.argmax(a)

1

In [ ]:
def game_with_comments(player1, player2, weight):
    position = np.zeros(64, dtype=int)
    
    while not check_win(position):

        # X - move
        res1 = player1.predict(position, weight)
        while position[np.argmax(res1)] != 0:
            res1[np.argmax(res1)] = float('-inf')
        position[np.argmax(res1)] = 1
        print("X -", np.argmax(res1))
        if check_win(position): # player1 won
            return

        # O - move
        res2 = player2.predict((-1) * position, weight)
        while position[np.argmax(res2)] != 0:
            res2[np.argmax(res2)] += float('-inf')
        position[np.argmax(res2)] = -1
        print("O -", np.argmax(res2))
    
    return

In [ ]:
def one_side_match(x_player, o_player):
    # game(x_player, o_player, -2)
    # game(x_player, o_player, -1)
    # game(x_player, o_player, -0.5)
    # game(x_player, o_player, 0.5)
    game(x_player, o_player, 1)
    # game(x_player, o_player, 2)

In [ ]:
def match(player1, player2):
    player1.games_played += 1
    player2.games_played += 1
    one_side_match(player1, player2)
    one_side_match(player2, player1)

In [ ]:
def copy_array(array):
    result = []
    for one in array:
        result.append(one)
    return result

In [ ]:
def tournament(players__):
    players = copy_array(players__)
    length = len(players)
    counter = 0
    while len(players) > length / 8:
        if players[counter].score < players[-1 - counter].score:
            players.remove(players[counter])
        else:
            players.remove(players[-1 - counter])
        counter += 1
    
    return players

In [ ]:
def mutation(person):
    person2 = copy.deepcopy(person)
    random_shit = randint(1, 3)
    if random_shit == 1:
        person2.add_node()
    if random_shit == 2:
        person2.add_connection()
    if random_shit == 3:
        person2.change_activation()
    return person2

In [ ]:
def one_iteration(players):
    for one in players:
        one.games_played = 0
        one.score = one.connections

    for one in players:
        for i in range(10):
            match(one, players[randint(0, population - 1)])

    players = sorted(players, key=lambda individ: individ.score)
    new_players = []
    counter = 0
    counter2 = 0
    while True:
        new_players.append(players[counter2])
        counter += 1
        for i in range(3):
            new_players.append(mutation(players[counter2]))
            counter += 1
            if counter == population:
                return new_players
        counter2 += 1

    return new_players

In [ ]:
def draw(individ):
    fig, ax = plt.subplots(1)
    pos = 0

    for layer in range(individ.max_layer + 1):
        for node in individ.nodes:
            if node.layer == layer:
                ax.scatter(layer, pos, c=colors[functions.index(node.foo)])
                node.pos = pos
                pos += 1
        pos = 0

    for node in individ.nodes:
        for one in node.next:
            ax.plot((node.layer, one.layer), (node.pos, one.pos), linewidth=0.2, color='blue')
    # plt.figure(figsize=(32, 18))
    plt.show()

In [ ]:
population = 100
iterations = 300

boys = []
for i in range(population):
    boys.append(Neyro())

# for one in boys:
#     draw(one)


for iteration in range(iterations):
    print("ITERATION", iteration, "...", len(boys))
    boys = one_iteration(boys)

game_with_comments(boys[0], boys[1], 1)

ITERATION 0 ... 100
ITERATION 1 ... 100
ITERATION 2 ... 100
ITERATION 3 ... 100


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app


ITERATION 4 ... 100
ITERATION 5 ... 100
ITERATION 6 ... 100
ITERATION 7 ... 100
ITERATION 8 ... 100
ITERATION 9 ... 100
ITERATION 10 ... 100
ITERATION 11 ... 100
ITERATION 12 ... 100
ITERATION 13 ... 100
ITERATION 14 ... 100
ITERATION 15 ... 100
ITERATION 16 ... 100
ITERATION 17 ... 100
ITERATION 18 ... 100
ITERATION 19 ... 100
ITERATION 20 ... 100
ITERATION 21 ... 100
ITERATION 22 ... 100
ITERATION 23 ... 100
ITERATION 24 ... 100
ITERATION 25 ... 100
ITERATION 26 ... 100
ITERATION 27 ... 100
ITERATION 28 ... 100
ITERATION 29 ... 100
ITERATION 30 ... 100
ITERATION 31 ... 100
ITERATION 32 ... 100
ITERATION 33 ... 100
ITERATION 34 ... 100
ITERATION 35 ... 100
ITERATION 36 ... 100
ITERATION 37 ... 100
ITERATION 38 ... 100
ITERATION 39 ... 100
ITERATION 40 ... 100
ITERATION 41 ... 100
ITERATION 42 ... 100
ITERATION 43 ... 100
ITERATION 44 ... 100
ITERATION 45 ... 100
ITERATION 46 ... 100
ITERATION 47 ... 100
ITERATION 48 ... 100
ITERATION 49 ... 100
ITERATION 50 ... 100
ITERATION 51 ... 10

In [ ]:
for one in boys:
    draw(one)